In [17]:
#Importing packages
from selenium import webdriver
#from pymongo import MongoClient
from collections import Counter
import pandas as pd
import json
import twint
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx
import nest_asyncio

In [18]:
driver = webdriver.Chrome()
driver.get('https://www.poynter.org/ifcn-covid-19-misinformation/?covid_countries=48837&covid_rating=0&covid_fact_checkers=0&orderby=views&order=DESC')

In [19]:
#client = MongoClient('mongodb://localhost:27017/')

In [20]:
news_list = []
user_list = []
username_list = []
edge_list = []
user_id_list = []
counter = 0

In [21]:
for page_num in range(2,5):
    for x in range(0,14):
        button = driver.find_elements_by_link_text('READ MORE')
        button[x].click()
        validity = driver.find_element_by_class_name('entry-title--red').text
        validity = validity[:-1]
        claim = driver.find_element_by_class_name('entry-title').text
        if not claim.strip():
            driver.back()
        
        else:
            news_list.append({'nodetype':"news", 'validity':validity.upper(), 'claim':claim, 'idnum':counter})
            counter = counter + 1
            driver.back()
        
    button = driver.find_element_by_link_text('{}'.format(page_num))
    button.click()

In [22]:
print(len(news_list))

42


In [23]:
driver.get('https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public/myth-busters')
content = driver.find_elements_by_css_selector('h2')

In [24]:
for con in content:
    if 'FACT: ' not in con.text:
        content.remove(con)
    else:
        news_list.append({'nodetype':"news", 'validity':"TRUE", 'claim':con.text.replace('FACT: ',''), 'idnum':counter})
        counter = counter + 1
print(len(news_list))

65


In [25]:
#print(news_list)
with open('news_list.json', 'w') as json_file:
    json.dump(news_list, json_file)   

In [26]:
G = nx.Graph()
a = 0
color_map = []
for news in news_list:
    G.add_node(a, n=news)
    color_map.append('blue')
    a = a + 1

In [ ]:
for news in news_list:
    nest_asyncio.apply()
    c = twint.Config()
    c.Search = news['claim']
    c.Limit = 10
    c.User_full = True
    c.Store_object = True
    c.Hide_output = True
    twint.run.Search(c)
    users = twint.output.users_list
    
    for user in users:
        username = user.username
        idnum = user.id
        name = user.name
        location = user.location
        num_followes = user.followers
        num_following = user.following
        date_joined = user.join_date
        #is_private = user.is_private
        #is_verified = user.is_verified
        #num_tweets = user.tweets
        
        if idnum in user_id_list:
            count = 0
            for x in user_list:
                if x['username'] == username:
                    count = x['idnum']
                    break
            edge_list.append({'type':'nu', 'edge':[count,news['idnum']]})    
        else:
            user_id_list.append(idnum)
            username_list.append(username)
            edge_list.append({'type':'nu', 'edge':[idnum,news['idnum']]})
            
            user_list.append({'type':'user', 'username':username, 'idnum':idnum, 'name':name, 'location':location, 'num_followes':num_followes, 'num_following':num_following, 'date_joined': date_joined})

            nest_asyncio.apply()
            c1 = twint.Config()
            c1.Search = news['claim']
            c1.Limit = 10
            c1.User_full = True
            c1.Store_object = True
            c1.Hide_output = True
            twint.run.Followers(c1)
            followers = twint.output.users_list

            for follow in followers:
                fusername = follow.username
                fidnum = follow.id
                fname = follow.name
                flocation = follow.location
                fnum_followes = follow.followers
                fnum_following = follow.following
                fdate_joined = follow.join_date
                #is_private = user.is_private
                #is_verified = user.is_verified
                #num_tweets = user.tweets

                if fidnum in user_id_list:
                    count = 0
                    for x in user_list:
                        if x['username'] == username:
                            count = x['idnum']
                            break
                    edge_list.append({'type':'uu', 'edge':[count,idnum]})    
                else:
                    user_id_list.append(idnum)
                    username_list.append(username)
                    edge_list.append({'type':'uu', 'edge':[idnum,fidnum]})

                    user_list.append({'type':'user', 'username':fusername, 'idnum':fidnum, 'name':fname, 'location':flocation, 'num_followes':fnum_followes, 'num_following':fnum_following, 'date_joined':fdate_joined}) 


CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 1.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 8.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 27.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 64.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 125.0 secs
CRITICAL:root:twint.run:Twint:Feed:noDatalist index out of range
sleeping for 216.0 secs


In [ ]:
with open('user_list.json', 'w') as json_file:
    json.dump(user_list, json_file)
with open('edge_list.json', 'w') as json_file:
    json.dump(edge_list, json_file)

In [ ]:
for user in user_list:
    G.add_node(a, u=user)
    color_map.append('green')
    a = a + 1

In [ ]:
print(G.number_of_nodes())
print(G.nodes[0])
print(G.number_of_edges())

In [ ]:
for edges in edge_list:
    G.add_edge(edges['edge'][0],edges['edge'][1])

In [ ]:
nx.draw_random(G, node_color=color_map, with_labels=True)
plt.savefig("network.png")
plt.show()

In [ ]:
print(nx.clustering(G))
print(nx.average_clustering(G))

In [ ]:
print(nx.reciprocity(G))